# Redirection Networks - Partitions and number of networks

### Marcos Costa Santos Carreira

## Imports

In [1]:
# import networkx as nx
# import random as rd
# from fractions import Fraction
# from math import factorial, log10
# from collections import Counter
from collections.abc import Iterable
from itertools import permutations
# from more_itertools import locate
import functools

In [2]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

In [3]:
# from tqdm.notebook import tqdm

In [4]:
# from mpl_toolkits.mplot3d import Axes3D

## Functions

### Partitions

#### Definitions

Generate partitions (code from Jerome Kelleher https://jeromekelleher.net/generating-integer-partitions.html )

In [5]:
def accel_asc(n):
    a = [0 for i in range(n + 1)]
    k = 1
    y = n - 1
    while k != 0:
        x = a[k - 1] + 1
        k -= 1
        while 2 * x <= y:
            a[k] = x
            y -= x
            k += 1
        l = k + 1
        while x <= y:
            a[k] = x
            a[l] = y
            yield a[:k + 2]
            x += 1
            y -= 1
        a[k] = x + y
        y = x + y - 1
        yield a[:k + 1]

We want restricted partitions of 2(N-1)-L with N-L elements rp(j)>=2

In [6]:
def res_par(n, l):
    lis_par = list(accel_asc(2 * (n - 1) - l))
    sel_par = [p for p in lis_par if (1 not in p) and (len(p) == (n - l))]
    return sel_par

In [7]:
def res_par_all(n):
    lis_par = [res_par(n, l) for l in range(2, n)]
    return lis_par

We now need to distribute L over these degrees, which means finding restricted partitions of 2(N-L-1) with N-L elements rp(j)<=degrees(j)

In [8]:
def fit_list(deg, p):
    return all([deg[j] >= p[j] for j in range(len(deg))])

In [9]:
def subtract_dict(deg, p):
    net_dict = {}
    for j in range(len(deg)):
        if deg[j] in net_dict:
            temp_list = net_dict[deg[j]] + [deg[j] - p[j]]
            temp_list.sort()
            net_dict[deg[j]] = temp_list
        else:
            net_dict[deg[j]] = [deg[j] - p[j]]
    return net_dict

In [10]:
def res_par_max_dict(n, l, deg):
    n_nucl = len(deg)
    lis_par = list(accel_asc(2 * (n - l - 1)))
    sel_par = [p for p in lis_par if (len(p) == n_nucl)]
    # Permutations!
    perm_par = list(set(permutations(sel_par[0])))
    if len(sel_par) > 1:
        for s in sel_par:
            perm_par = list(set(perm_par + list(set(permutations(s)))))
    fit_par = [p for p in perm_par if fit_list(deg, p)]
    num_list = []
    for p in fit_par:
        new_elem = subtract_dict(deg, p)
        if new_elem not in num_list:
            num_list = num_list + [new_elem]
    return num_list

In [11]:
def flatten(xs):
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x

In [12]:
def res_par_max_list(n, l, deg):
    n_nucl = len(deg)
    lis_par = list(accel_asc(2 * (n - l - 1)))
    sel_par = [p for p in lis_par if (len(p) == n_nucl)]
    # Permutations!
    perm_par = list(set(permutations(sel_par[0])))
    if len(sel_par) > 1:
        for s in sel_par:
            perm_par = list(set(perm_par + list(set(permutations(s)))))
    fit_par = [p for p in perm_par if fit_list(deg, p)]
    num_list = []
    for p in fit_par:
        new_elem = subtract_dict(deg, p)
        if new_elem not in num_list:
            num_list = num_list + [new_elem]
    return [list(flatten(list(elem.values()))) for elem in num_list]

In [13]:
def res_numdeg_all(n):
    deg_list = res_par_all(n)
    numdeg_list = [[[deg, res_par_max_list(n, n - len(deg), deg)] for deg in deg_l] for deg_l in deg_list]
    return numdeg_list

In [14]:
def res_numdeg_all_count(n):
    deg_list = res_par_all(n)
    numdeg_list = [sum([len(res_par_max_list(n, n - len(deg), deg)) for deg in deg_l]) for deg_l in deg_list]
    return numdeg_list

#### Examples

Restricted partitions for n=8

In [15]:
res_par_all(8)

[[[2, 2, 2, 2, 2, 2]],
 [[2, 2, 2, 2, 3]],
 [[2, 2, 2, 4], [2, 2, 3, 3]],
 [[2, 2, 5], [2, 3, 4], [3, 3, 3]],
 [[2, 6], [3, 5], [4, 4]],
 [[7]]]

Restricted partitions and their admissable kernels for n=8

In [16]:
res_numdeg_all(8)

[[[[2, 2, 2, 2, 2, 2], [[0, 0, 0, 0, 1, 1]]]],
 [[[2, 2, 2, 2, 3], [[0, 1, 1, 1, 0], [0, 0, 1, 1, 1], [0, 0, 0, 1, 2]]]],
 [[[2, 2, 2, 4], [[0, 1, 1, 2], [0, 0, 1, 3], [1, 1, 1, 1]]],
  [[2, 2, 3, 3], [[0, 1, 1, 2], [1, 1, 0, 2], [0, 0, 2, 2], [1, 1, 1, 1]]]],
 [[[2, 2, 5], [[0, 1, 4], [1, 1, 3]]],
  [[2, 3, 4], [[1, 1, 3], [0, 2, 3], [1, 2, 2]]],
  [[3, 3, 3], [[1, 2, 2]]]],
 [[[2, 6], [[1, 5]]], [[3, 5], [[2, 4]]], [[4, 4], [[3, 3]]]],
 [[[7], [[7]]]]]

Number of admissable kernels for n=8

In [17]:
res_numdeg_all_count(8)

[1, 3, 7, 6, 3, 1]

Totals of admissable kernels, n from 3 to 12

In [18]:
%%time

[[n, sum(res_numdeg_all_count(n))] for n in range(3, 12 + 1)]

CPU times: user 4.04 s, sys: 11.4 ms, total: 4.05 s
Wall time: 4.05 s


[[3, 1],
 [4, 2],
 [5, 3],
 [6, 6],
 [7, 11],
 [8, 21],
 [9, 38],
 [10, 70],
 [11, 123],
 [12, 216]]

### Networks

#### Number of free trees with n unlabeled nodes (A000055)

In [19]:
# del s55

In [20]:
# del c55

In [21]:
# del cs55

In [22]:
@functools.cache
def s55(n, k):
    if n < 2 * k:
        return c55(n + 1 - k)
    else:
        return c55(n + 1 - k) + s55(n - k, k)

In [23]:
@functools.cache
def c55(n):
    if n == 1:
        return 1
    else:
        ans = 0
        for i in range(1, n):
            ans = ans + c55(i) * s55(n - 1, i) * i
        return ans // (n - 1)

In [24]:
@functools.cache
def cs55(n):
    ans = c55(n)
    for j in range(1, n // 2 + 1):
        ans = ans - c55(j) * c55(n - j)
    if n % 2 == 0:
        return ans + c55(n // 2) * (c55(n // 2) + 1) // 2
    else:
        return ans

In [25]:
[[n, cs55(n), sum(res_numdeg_all_count(n))] for n in range(3, 12 + 1)]

[[3, 1, 1],
 [4, 2, 2],
 [5, 3, 3],
 [6, 6, 6],
 [7, 11, 11],
 [8, 23, 21],
 [9, 47, 38],
 [10, 106, 70],
 [11, 235, 123],
 [12, 551, 216]]

In [26]:
[[n, cs55(n)] for n in range(1, 21)]

[[1, 1],
 [2, 1],
 [3, 1],
 [4, 2],
 [5, 3],
 [6, 6],
 [7, 11],
 [8, 23],
 [9, 47],
 [10, 106],
 [11, 235],
 [12, 551],
 [13, 1301],
 [14, 3159],
 [15, 7741],
 [16, 19320],
 [17, 48629],
 [18, 123867],
 [19, 317955],
 [20, 823065]]

In [27]:
%%time

cs55(100)

CPU times: user 8.42 ms, sys: 3.49 ms, total: 11.9 ms
Wall time: 10.8 ms


630134658347465720563607281977639527019590